
# Browser Attribute Fingerprinting Analysis

### What is fingerprinting? 
[Fingerprinting](https://en.wikipedia.org/wiki/Device_fingerprint) is the process of creating a unique identifier based off of some characteristics of your hardware, operating system and browser.

###  What is Browser Attribute Fingerprinting?
Browser Attribute Fingerprintingis the process of creating a unique identifier on the basis of browser characteristics. It is a powerful method that websites use to collect information about your browser using information collected from active plugins, browser version, OS version, timezone, language, screen resolution and various other active settings.

These data points might seem generic at first and don’t necessarily look tailored to identify one specific person. However, there’s a significantly small chance for another user to have 100% matching browser information. .

The EFF (Electronic Frontier Foundation) published a study [Is Every Browser Unique?]().Results Fom The Panopticlick Experiment found that, over their study of around 1 million visits to their study website, 83.6% of the browsers seen had a unique fingerprint; among those with Flash or Java enabled, 94.2%.<b>This does not include cookies!</b> 

This shows that the overwhelming majority of Internet users could be uniquely fingerprinted and tracked using only the configuration and version information that their browsers make available to websites. These types of system information should be regarded as identifying, in much the same way that cookies, IP addresses, and supercookies are. 

An example shared in the paper by EFF tells, A browser with JavaScript disabled will record default values for video, plugins, fonts and supercookies, so the presence of these measurements indicates that JavaScript is active. More subtly, browsers with a Flash blocking add-on installed show Flash in the plugins list, but fail to obtain a list of system font via Flash, thereby creating a distinctive fingerprint, even though neither measurement (plugins, fonts) explicitly detects the Flash blocker. Similarly many browsers with forged User Agent strings are distinguished because the other measurements do not comport with the User Agent. This manifests that even when we are using flash blocking add-ons or disabling javascript to enhance our security, such acts in fact causing our browser to be more prone to fingerprintng.


If we consider the case of Canvas Fingerprinting and how it works,

Canvas Fingerprinting is one of a number of browser fingerprinting techniques for tracking online users that allow websites to identify and track visitors using the HTML5 canvas element instead of browser cookies or other similar means.

As described by Acar et. al. in The Web never forgets: Persistent tracking mechanisms in the wild,

1. Fingerprinting script first draws text with the font and size of its choice and adds background colors

2. Next, the script calls Canvas API’s ToDataURL method to get the canvas pixel data in dataURL format, which is basically a Base64 encoded representation of the binary pixel data. 

3. Finally, the script takes the hash of the text-encoded pixel data, which serves as the fingerprint.

As we see the hash of the text encoded pixel data serves as the fingerprint while considering the Canvas Fingerprint, What will be considered as a fingerprint in the case of Browser Attribute Fingerprinting?

Let's look at the methodology implemented by EFF in their paper[] () to find out. They implemented a browser fingerprinting algorithm by collecting a number ofcommonly and less-commonly known characteristics that browsers make avalable to websites. Some of these can be inferred from the content of simple, static HTTP requests; others were collected by AJAX2 . They grouped the measurements into eight separate strings. <b>The fingerprint was essentially the concatenation of these strings.</b>

You can test how easy your device is to identify at the website [amiunique.org](https://amiunique.org/), 
which has collected over thousands of different device fingerprints for research purposes. The site will render your device fingerprint and assess how “unique” you are based on the following inputs:

    - the User agent header
    - the Accept header
    - the Connection header
    - the Encoding header
    - the Language header
    - the list of plugins
    - the platform
    - the cookies preferences (allowed or not)
    - the Do Not Track preferences (yes, no or not communicated)
    - the timezone
    - the screen resolution and its color depth
    - the use of local storage
    - the use of session storage
    - a picture rendered with the HTML Canvas element
    - a picture rendered with WebGL
    - the presence of AdBlock
    - the list of fonts

## What is our objective?
Our main objective here is to build a heuristic for browser attribute fingerprinting that pulls out these scripts?

### How I intend to accomplish the obective

We know, there are some scripts that we can pick out by name that are doing browser attribute fingerprinting:

- Scripts with hs-analytics in the script_url
- Scripts with /akam/ in the script_url
- The fingerprintjs scripts

We will start by analyzing these scripts. Looking at the sybmol we get an insight related to the purpose behind the script being called. This can be emphasised as there is a direct relation between the passed to the symbol and the value being passed to the script. For instance, if the "symbol" is "window.navigator.userAgent" then the value passedto the called script will be the user agent of the client which can be seen from the value column corresponding to the script. So, we will look for the symbols present in these three scripts along with the count of the scripts for each symbol.

From the set of symbols found in the hs-analytics, akam, fingerprintjs scripts we will look for those whose coressponding scripts which can probably be contributing to browser attribute fingerprinting. We can get an idea of the potential symbols from the list of inputs from amiunique.org(mentioned above).

Then we compute a dataframe combining the scripts we have got from the potential symbols. This will consist of the possible browser attributing scripts. A lot of work needs to be done to on this dataframe to make sure which scripts are actually contributing to browser attribute fingerprinting as there can be cases of false positives.

Looking at the operation of the script, if  it is a get operation the chances of the script contributing to fingerprinting increases as it s collecting some information relating to the client. Also, majority of the times the hs-analytics, akam, fingerprintjs scripts are performing the get operation.

One intresting behaviour shown by the three fingerprinting scripts is that a large number of calls are made during similar time.










In [2]:
import dask.dataframe as dd
import pandas as pd

from dask.distributed import Client

Client(memory_limit='16GB')

/home/soumya/.conda/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/soumya/.conda/envs/overscripted/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:39769 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 64.00 GB


In [3]:
df = dd.read_parquet(
    '/home/soumya/Dataset/sample_0.parquet', engine='pyarrow',
    columns=[ 'symbol', 'location', 'script_url', 'time_stamp', 'value','value_len','operation','in_iframe']
)

In [4]:
 pd.options.display.max_rows = None
# Do not truncate cell values
 pd.set_option('display.max_colwidth', -1)

##  Get hs-analytics scripts

In [5]:
hs = df[df.script_url.str.contains('hs-analytics')]
hs.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00a42e801381714f82b0a3b03e22712c3483c8f4d1c12468fac52503.json__454,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/1513389300000/1959817.js,2017-12-16 01:53:37.180,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=1406116232%7CMCIDTS%7C17517%7CMCMID%7C59012494228105667413882085630674972173%7CMCAAMLH-1513994010%7C9%7CMCAAMB-1513994010%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1513396410s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-17524%7CvVersion%7C2.5.0; AMCVS_AAB73BC75245B44A0A490D4D%40AdobeOrg=1; check=true; dmdbase_cdc=DBSET; _ga=GA1.2.563481909.1513389211; _gid=GA1.2.504612378.1513389211; _gat=1; acn_coach_mark=1; mbox=session#8b9fb7676a9249bbb40a83194f3d552a#1513391073|PC#8b9fb7676a9249bbb40a83194f3d552a.28_55#1576634013; s_nr=1513389212451-New; _ss_gpv_pn=acn%3Aca-en%3Apage%3Aterms-of-use; _ss_vo_psc=topofpage; s_cc=true; __qca=P0-999367461-1513389212783; spid=3D0BEA26-931B-4B95-B7AE-412947759630; sp_ssid=1513389213145; sp_sync_ssid=1513389213147; _mkto_trk=id:729-QXG-911&token:_mch-accenture.com-1513389214143-11792; __srret=1; sp_apnxid=3492364477833627885; __srui=edc7803a-e203-11e7-a101-22000bdd47f7,972,get,False
1_00a42e801381714f82b0a3b03e22712c3483c8f4d1c12468fac52503.json__455,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/1513389300000/1959817.js,2017-12-16 01:53:37.181,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=1406116232%7CMCIDTS%7C17517%7CMCMID%7C59012494228105667413882085630674972173%7CMCAAMLH-1513994010%7C9%7CMCAAMB-1513994010%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1513396410s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-17524%7CvVersion%7C2.5.0; AMCVS_AAB73BC75245B44A0A490D4D%40AdobeOrg=1; check=true; dmdbase_cdc=DBSET; _ga=GA1.2.563481909.1513389211; _gid=GA1.2.504612378.1513389211; _gat=1; acn_coach_mark=1; mbox=session#8b9fb7676a9249bbb40a83194f3d552a#1513391073|PC#8b9fb7676a9249bbb40a83194f3d552a.28_55#1576634013; s_nr=1513389212451-New; _ss_gpv_pn=acn%3Aca-en%3Apage%3Aterms-of-use; _ss_vo_psc=topofpage; s_cc=true; __qca=P0-999367461-1513389212783; spid=3D0BEA26-931B-4B95-B7AE-412947759630; sp_ssid=1513389213145; sp_sync_ssid=1513389213147; _mkto_trk=id:729-QXG-911&token:_mch-accenture.com-1513389214143-11792; __srret=1; sp_apnxid=3492364477833627885; __srui=edc7803a-e203-11e7-a101-22000bdd47f7,972,get,False
1_00a42e801381714f82b0a3b03e22712c3483c8f4d1c12468fac52503.json__456,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/1513389300000/1959817.js,2017-12-16 01:53:37.182,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=1406116232%7CMCIDTS%7C17517%7CMCMID%7C59012494228105667413882085630674972173%7CMCAAMLH-1513994010%7C9%7CMCAAMB-1513994010%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1513396410s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-17524%7CvVersion%7C2.5.0; AMCVS_AAB73BC75245B44A0A490D4D%40AdobeOrg=1; check=true; dmdbase_cdc=DBSET; _ga=GA1.2.563481909.1513389211; _gid=GA1.2.504612378.1513389211; _gat=1; acn_coach_mark=1; mbox=session#8b9fb7676a9249bbb40a83194f3d552a#1513391073|PC#8b9fb7676a9249bbb40a83194f3d552a.28_55#1576634013; s_nr=1513389212451-New; _ss_gpv_pn=acn%3Aca-en%3Apage%3Aterms-of-use; _ss_vo_psc=topofpage; s_cc=true; __qca=P0-999367461-1513389212783; spid=3D0BEA26-931B-4B95-B7AE-412947759630; sp_ssid=1513389213145; sp_sync_ssid=1513389213147; _mkto_trk=id:729-QXG-911&token:_mch-accenture.com-1513389214143-11792; __srret=1; sp_apnxid=3492364477833627885; __srui=edc7803a-e203-11e7-a101-22000bdd47f7,972,get,False
1_00a42e801381714f82b0a3b03e22712c3483c8f4d1c12468fac52503.json__457,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/1513389300000/1959817.js,2017-12-16 01:53:37.182,"hsfirstvisit=;expires=Sun, 01 Jan 2006 00:00:00 GMT

In [5]:
len(hs)

13848

In [6]:
hs.script_url.nunique().compute()

230

## Symbol counts in scripts having hs-analytics in script_url

In [8]:
hsSymbols=hs.symbol.value_counts().compute()
hsSymbols

window.document.cookie            7696
window.navigator.language         1414
window.screen.colorDepth          1052
window.navigator.userAgent        921 
window.localStorage               692 
window.sessionStorage             690 
window.navigator.platform         690 
window.navigator.doNotTrack       690 
window.Storage.getItem            2   
window.navigator.cookieEnabled    1   
Name: symbol, dtype: int64

##  Get akam scripts

In [ ]:
akam = df[df.script_url.str.contains('/akam/')]
akam.head()

In [10]:
len(akam)

60975

In [11]:
akam.script_url.nunique().compute()

853

## Symbol counts in scripts having akam in script_url

In [12]:
akamSymbols=akam.symbol.value_counts().compute()
akamSymbols

window.navigator.plugins[Shockwave Flash].description                    3312
CanvasRenderingContext2D.fillStyle                                       3234
window.navigator.plugins[Shockwave Flash].name                           2484
window.navigator.userAgent                                               2484
window.navigator.plugins[Shockwave Flash].length                         2484
CanvasRenderingContext2D.fillRect                                        2406
window.Storage.setItem                                                   1656
window.navigator.appCodeName                                             1656
window.navigator.language                                                1656
window.Storage.removeItem                                                1656
window.navigator.appName                                                 1656
HTMLCanvasElement.getContext                                             1656
window.navigator.appVersion                                     

## Get fingerprintjs2 scripts

In [8]:
fp2 = df[df.script_url.str.contains('fingerprint2')]
fp2.get_partition(1).compute()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__9,window.navigator.userAgent,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DAKAOjssr2XFA691c63VXOl1VoxPcB_Vx4veVq-8KSv1gS0NoiWNNR6krGgPv6Kh4g5CtIObEcb6_wdOZttY54GFIBYodJjSvPwBx7S1SxsmItaM4GOj85fbnk35INePbARHMidvSSw_lfwq9gN393H-wSUuwfTE3GitZ3O74BHGXgp1UY-Gl-jV1IrX6WmItdD-LtGY3j3UAjS-od0A-fKE9bnkSTnb25Rj0DaY---w6k1weLASF9ztEF0IWu_7gH45nhO0h5SNVqx9XQoapuEwv%2526sig%253DCg0ArKJSzOYb-kOnl2P_EAE%2526urlfix%253D1%2526adurl%253D,https://img.mediaplex.com/0/711/fingerprint2.js,2017-12-16 08:10:03.982,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__10,window.navigator.language,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DAKAOjssr2XFA691c63VXOl1VoxPcB_Vx4veVq-8KSv1gS0NoiWNNR6krGgPv6Kh4g5CtIObEcb6_wdOZttY54GFIBYodJjSvPwBx7S1SxsmItaM4GOj85fbnk35INePbARHMidvSSw_lfwq9gN393H-wSUuwfTE3GitZ3O74BHGXgp1UY-Gl-jV1IrX6WmItdD-LtGY3j3UAjS-od0A-fKE9bnkSTnb25Rj0DaY---w6k1weLASF9ztEF0IWu_7gH45nhO0h5SNVqx9XQoapuEwv%2526sig%253DCg0ArKJSzOYb-kOnl2P_EAE%2526urlfix%253D1%2526adurl%253D,https://img.mediaplex.com/0/711/fingerprint2.js,2017-12-16 08:10:03.984,en-US,5,get,True
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__11,window.screen.colorDepth,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DAKAOjssr2XFA691c63VXOl1VoxPcB_Vx4veVq-8KSv1gS0NoiWNNR6krGgPv6Kh4g5CtIObEcb6_wdOZttY54GFIBYodJjSvPwBx7S1SxsmItaM4GOj85fbnk35INePbARHMidvSSw_lfwq9gN393H-wSUuwfTE3GitZ3O74BHGXgp1UY-Gl-jV1IrX6WmItdD-LtGY3j3UAjS-od0A-fKE9bnkSTnb25Rj0DaY---w6k1weLASF9ztEF0IWu_7gH45nhO0h5SNVqx9XQoapuEwv%2526sig%253DCg0ArKJSzOYb-kOnl2P_EAE%2526urlfix%253D1%2526adurl%253D,https://img.mediaplex.com/0/711/fingerprint2.js,2017-12-16 08:10:03.984,24,2,get,True
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__12,window.sessionStorage,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DAKAOjssr2XFA691c63VXOl1VoxPcB_Vx4veVq-8KSv1gS0NoiWNNR6krGgPv6Kh4g5CtIObEcb6_wdOZttY54GFIBYodJjSvPwBx7S1SxsmItaM4GOj85fbnk35INePbARHMidvSSw_lfwq9gN393H-wSUuwfTE3GitZ3O74BHGXgp1UY-Gl-jV1IrX6WmItdD-LtGY3j3UAjS-od0A-fKE9bnkSTnb25Rj0DaY---w6k1weLASF9ztEF0IWu_7gH45nhO0h5SNVqx9XQoapuEwv%2526sig%253DCg0ArKJSzOYb-kOnl2P_EAE%2526urlfix%253D1%2526adurl%253D,https://img.mediaplex.com/0/711/fingerprint2.js,2017-12-16 08:10:03.985,{},2,get,True
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__13,window.localStorage,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DAKAOjssr2XFA691c63VXOl1VoxPcB_Vx4veVq-8KSv1gS0NoiWNNR6krGgPv6Kh4g5CtIObEcb6_wdOZttY54GFIBYodJjSvPwBx7S1SxsmItaM4GOj85fbnk35INePbARHMidvSSw_lfwq9gN393H-wSUuwfTE3GitZ3O74BHGXgp1UY-Gl-jV1IrX6WmItdD-LtGY3j3UAjS-od0A-fKE9bnkSTnb25Rj0DaY---w6k1weLASF9ztEF0IWu_7gH45nhO0h5SNVqx9XQoapuEwv%2526sig%253DCg0ArKJSzOYb-kOnl2P_EAE%2526urlfix%253D1%2526adurl%253D,https://img.mediaplex.com/0/711/fingerprint2.js,2017-12-16 08:10:03.985,{},2,get,True
1_00f21360a314a761888a7ec4341a0a79dced91747d73f84edd1fd843.json__14,window.navigator.platform,https://rover.ebay.com/ar/1/152815/4?mpt=696349298&siteid=77&icep_siteid=77&ipn=admain2&adtype=2&size=300x250&pgroup=432317&mpvc=https://adclick.g.doubleclick.net/pcs/click%253Fxai%253DA

In [14]:
len(fp2)

10649

In [15]:
fp2.script_url.nunique().compute()

216

## Symbol counts in scripts having fingerprintjs in script_url

In [16]:
fp2Symbols=fp2.symbol.value_counts().compute()
fp2Symbols


CanvasRenderingContext2D.fillStyle                                    1031
window.navigator.userAgent                                            825 
HTMLCanvasElement.getContext                                          767 
CanvasRenderingContext2D.arc                                          615 
window.navigator.platform                                             557 
window.navigator.plugins[Shockwave Flash].length                      540 
CanvasRenderingContext2D.fill                                         492 
CanvasRenderingContext2D.rect                                         416 
window.navigator.appName                                              372 
window.navigator.doNotTrack                                           364 
window.navigator.language                                             343 
window.navigator.languages                                            258 
CanvasRenderingContext2D.fillText                                     246 
CanvasRenderingContext2D.

## Getting symbol specific scripts


In [9]:
ua =df[df.symbol == 'window.navigator.userAgent']
ua.head()

/home/soumya/.local/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00006011493ed94fb8010cead84ee610cdbece5de961abe96f0cba54.json__2,window.navigator.userAgent,https://vk.com/widget_comments.php?app=2297596&width=100%25&_ver=1&limit=10&height=0&mini=auto&norealtime=0&page=0&status_publish=0&attach=*&url=https%3A%2F%2Fwww.gl5.ru%2Flena-katina-silent-hills.html&title=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&description=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&image=&startWidth=600&referrer=&16060b4b60d,https://vk.com/js/al/aes_light.js?592436914,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_00006b1ad6ce7a5b2b81a8ef96f0778853ea6d81a97693d85a52790c.json__0,window.navigator.userAgent,https://pos.baidu.com/s?hei=70&wid=670&di=u3130587&ltu=https%3A%2F%2Fielts.liuxue86.com%2Fi%2F3040174.html&dis=0&pss=1353x4446&dri=0&ccd=24&ant=0&pis=-1x-1&cce=true&ti=%E5%88%A9%E7%94%A8%E7%B2%BE%E8%AF%BB%E5%88%86%E6%9E%90%E9%9B%85%E6%80%9D%E9%98%85%E8%AF%BB%E6%AE%B5%E8%90%BD&tpr=1513383844066&exps=111000&cfv=0&par=1366x768&dtm=HTML_POST&psr=1366x768&ps=3274x176&ari=2&cja=false&chi=1&prot=2&dai=8&tcn=1513383844&cec=UTF-8&drs=3&cpl=1&dc=2&cdo=-1&tlm=1513383844&pcs=1353x697&cmi=2&col=en-US,https://cpro.baidustatic.com/cpro/ui/noexpire/js/4.0.0/adClosefeedbackUpgrade.min.js,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__43,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.721,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__46,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.722,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__68,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:22.020,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False


In [18]:
ua.script_url.nunique().compute()



107415

In [10]:
language = df[(df.symbol == 'window.navigator.language') | (df.symbol == 'window.navigator.languages')]
language.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__39,window.navigator.language,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.717,en-US,5,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__146,window.navigator.language,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.667,en-US,5,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__147,window.navigator.languages,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.667,"[""en-US"",""en""]",14,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__210,window.navigator.language,https://www.coches.net/fiat/segunda-mano/,https://jssdk.pulse.schibsted.com/autoTrackerCochesnet.min.js,2017-12-16 23:20:04.032,en-US,5,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__211,window.navigator.language,https://www.coches.net/fiat/segunda-mano/,https://jssdk.pulse.schibsted.com/autoTrackerCochesnet.min.js,2017-12-16 23:20:04.033,en-US,5,get,False


In [22]:
language.script_url.nunique().compute()

8338

In [11]:
platform = df[df.symbol == 'window.navigator.platform']
platform.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__68,window.navigator.platform,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://uaction.alicdn.com/js/ua.js,2017-12-16 00:10:56.937,Linux x86_64,12,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__102,window.navigator.platform,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/aliww/web.ww/scripts/webww.js,2017-12-16 00:10:58.358,Linux x86_64,12,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__138,window.navigator.platform,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.660,Linux x86_64,12,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__234,window.navigator.platform,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.492,Linux x86_64,12,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__235,window.navigator.platform,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.493,Linux x86_64,12,get,False


In [21]:
platform.script_url.nunique().compute()

13935

In [12]:
cookieenabled = df[df.symbol == 'window.navigator.cookieEnabled']
cookieenabled.head()


,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__205,window.navigator.cookieEnabled,https://www.coches.net/fiat/segunda-mano/,https://jssdk.pulse.schibsted.com/autoTrackerCochesnet.min.js,2017-12-16 23:20:04.018,true,4,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__222,window.navigator.cookieEnabled,https://www.coches.net/fiat/segunda-mano/,https://script.hotjar.com/modules-526d80f8c01454f84b75838f21c8706e.js,2017-12-16 23:20:04.434,true,4,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__345,window.navigator.cookieEnabled,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:05.186,true,4,get,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__124,window.navigator.cookieEnabled,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,https://mc.yandex.ru/metrika/watch.js,2017-12-16 05:25:22.376,true,4,get,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__192,window.navigator.cookieEnabled,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,https://mc.yandex.ru/metrika/watch.js,2017-12-16 05:25:22.428,true,4,get,False


In [23]:
cookieenabled.script_url.nunique().compute()

10140

In [13]:
donottrack = df[df.symbol == 'window.navigator.doNotTrack']
donottrack.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__145,window.navigator.doNotTrack,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.667,unspecified,11,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__236,window.navigator.doNotTrack,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.493,unspecified,11,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__237,window.navigator.doNotTrack,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.493,unspecified,11,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__296,window.navigator.doNotTrack,https://www.coches.net/fiat/segunda-mano/,https://script.hotjar.com/modules-526d80f8c01454f84b75838f21c8706e.js,2017-12-16 23:20:04.809,unspecified,11,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__305,window.navigator.doNotTrack,https://www.coches.net/fiat/segunda-mano/,https://script.hotjar.com/modules-526d80f8c01454f84b75838f21c8706e.js,2017-12-16 23:20:04.814,unspecified,11,get,False


In [25]:
donottrack.script_url.nunique().compute()

3657

In [14]:
localStorage = df[df.symbol == 'window.localStorage']
localStorage.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__73,window.localStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.137,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__77,window.localStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.140,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__81,window.localStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.141,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__87,window.localStorage,https://www.canada.ca/en/services.html,https://www.canada.ca/etc/designs/canada/wet-boew/js/wet-boew.min.js,2017-12-16 02:54:22.458,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__88,window.localStorage,https://www.canada.ca/en/services.html,https://www.canada.ca/etc/designs/canada/wet-boew/js/wet-boew.min.js,2017-12-16 02:54:22.459,{},2,get,False


In [27]:
localStorage.script_url.nunique().compute()

25490

In [15]:
sessionStorage = df[df.symbol == 'window.sessionStorage']
sessionStorage.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__75,window.sessionStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.139,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__79,window.sessionStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.140,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__83,window.sessionStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.321,{},2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__85,window.sessionStorage,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/satelliteLib-78cf42deb149c9766cbaaa6151e252b9b67c0200.js,2017-12-16 02:54:22.324,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site\"":\""0.15\"",\""scopeHat\"":\""Page\"",\""tbl_url_whitelist\"":[{\""ID\"":\""1\"",\""url\"":\""^www\\\\.canada\\\\.ca\\\\/\"",\""settings_link\"":\""1\""},{\""ID\"":\""2\"",\""url\"":\""^canada-stage-aem62s3\\\\.adobecqms\\\\.net\\\\/\"",\""settings_link\"":\""1\""},{\""ID\"":\""4\"",\""settings_link\"":\""1\""}],\""persistent_cookie_duration\"":\""15\""},\""surveys\"":[{\""id\"":\""93\"",\""title-en\"":\""Visitor survey\"",\""title-fr\"":\""Sondage des visiteurs\"",\""body-en\"":\""<p>We want to know more about you!</p> <p>Please take a few moments to provide some anonymous information about yourself. This will help us to improve the Canadian Armed Forces recruiting process and has NO impact on your current or future application. </p><p><p class=\\\""mrgn-bttm-0\\\"">Choosing “Yes” will open a new window where you can complete the survey and then return to the page and continue your visit to Canada.ca/Forces-Jobs.</p>\"",\""body-fr\"":\""<p>Nous voulons en savoir plus à votre sujet!</p><p>Veuillez prendre un moment pour nous fournir des renseignements anonymes à propos de vous. Cette information nous aidera à améliorer le processus de recrutement des Forces armées canadiennes et n’a AUCUNE incidence sur votre processus de demande actuelle ou à venir. </p><p><p class=\\\""mrgn-bttm-0\\\"">En choisissant « Oui », une nouvelle fenêtre s’ouvrira et vous pourrez répondre au sondage. Ensuite, vous pouvez retourner à la page et continuer de consulter le site Canada.ca/Forces-Emplois.</p>\"",\""link-en\"":\""https://survey-sondage.forces.gc.ca/snapwebhost/s.asp?k=150532412084\"",\""link-fr\"":\""https://survey-sondage.forces.gc.ca/snapwebhost/s.asp?k=150532412084\"",\""start_date_time\"":\""2017-11-20T16:00:00\"",\""end_date_time\"":\""2018-02-08T23:59:00\"",\""type\"":\""Page\"",\""probability\"":\""1\"",\""uniqueTitle\"":\""DND - Page Canadian Armed Forces Prospect Survey - 2017-11-21\"",\""yes-en\"":\""<strong>Yes<span class=\\\""wb-inv\\\"">I will take the survey </span><span class=\\\""wb-inv\\\"">.</span></strong>\"",\""yes-fr\"":\""<strong>Oui<span class=\\\""wb-inv\\\"">je vais participer au sondage</span><span class=\\\""wb-inv\\\"">.</span></strong>\"",\""no-en\"":\""No <span class=\\\""wb-inv\\\"">I do not want to take the survey, </span><span class=\\\""wb-inv\\\"">.</span>\"",\""no-fr\"":\""Non<span class=\\\""wb-inv\\\"">je ne veux pas participer au sondage </span><span class=\\\""wb-inv\\\"">.</span>\"",\""close-en\"":\""Close: Visitor survey (escape key)\"",\""close-fr\"":\""Fermer : Sondage des visiteurs (touche d'échappement)\"",\""survey_name\"":\""DND - Canadian Armed Forces Prospect Survey - Page\"",\""tbl_survey_urls\"":[{\""ID\

In [29]:
sessionStorage.script_url.nunique().compute()

12232

In [16]:
plugins = df[(df.symbol == 'window.navigator.plugins[Shockwave Flash].description') | 
             (df.symbol == 'window.navigator.plugins[Shockwave Flash].name') | 
             (df.symbol == 'window.navigator.plugins[Shockwave Flash].filename') |
             (df.symbol == 'window.navigator.plugins[Shockwave Flash].length') |  
             (df.symbol == 'window.navigator.plugins[Shockwave Flash].version')]
plugins.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__37,window.navigator.plugins[Shockwave Flash].name,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.716,Shockwave Flash,15,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__38,window.navigator.plugins[Shockwave Flash].description,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.716,Shockwave Flash 28.0 r0,23,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__140,window.navigator.plugins[Shockwave Flash].name,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.665,Shockwave Flash,15,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__141,window.navigator.plugins[Shockwave Flash].name,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.665,Shockwave Flash,15,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__142,window.navigator.plugins[Shockwave Flash].name,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://g.alicdn.com/secdev/sufei_data/3.2.2/index.js,2017-12-16 00:11:00.666,Shockwave Flash,15,get,False


In [31]:
plugins.script_url.nunique().compute()


14760

In [17]:
mimeTypes = df[(df.symbol == 'window.navigator.mimeTypes[application/x-shockwave-flash].type') | 
             (df.symbol == 'window.navigator.mimeTypes[application/futuresplash].type') | 
             (df.symbol == 'window.navigator.mimeTypes[application/x-shockwave-flash].suffixes') |
             (df.symbol == 'window.navigator.mimeTypes[application/futuresplash].suffixes') |  
             (df.symbol == 'window.navigator.mimeTypes[application/x-shockwave-flash].description')|
             (df.symbol == 'window.navigator.mimeTypes[application/futuresplash].description')]
mimeTypes.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__111,window.navigator.mimeTypes[application/futuresplash].type,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,2017-12-16 00:10:58.955,application/futuresplash,24,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__112,window.navigator.mimeTypes[application/x-shockwave-flash].type,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,2017-12-16 00:10:58.955,application/x-shockwave-flash,29,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__113,window.navigator.mimeTypes[application/futuresplash].type,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,2017-12-16 00:10:58.955,application/futuresplash,24,get,False
1_0001e211e7dd3877ff0243ae0ba0d1f6f9963ca210656cbd8c53681a.json__114,window.navigator.mimeTypes[application/x-shockwave-flash].type,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,https://maniform.world.tmall.com/category-1282610376.htm?search=y&catName=%C8%C8%CF%FA%B1%AC%BF%EE,2017-12-16 00:10:58.955,application/x-shockwave-flash,29,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__244,window.navigator.mimeTypes[application/x-shockwave-flash].type,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.495,application/x-shockwave-flash,29,get,False


In [34]:
mimeTypes.script_url.nunique().compute()


3192

In [18]:
colorDepth = df[df.symbol == 'window.screen.colorDepth']
colorDepth.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__36,window.screen.colorDepth,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.716,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__48,window.screen.colorDepth,https://www.coches.net/fiat/segunda-mano/,https://tags.tiqcdn.com/utag/schibsted/coches.net/prod/utag.js,2017-12-16 23:20:02.891,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__49,window.screen.colorDepth,https://www.coches.net/fiat/segunda-mano/,https://tags.tiqcdn.com/utag/schibsted/coches.net/prod/utag.js,2017-12-16 23:20:02.892,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__300,window.screen.colorDepth,https://www.coches.net/fiat/segunda-mano/,https://script.hotjar.com/modules-526d80f8c01454f84b75838f21c8706e.js,2017-12-16 23:20:04.813,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__367,window.screen.colorDepth,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:05.191,24,2,get,False


In [56]:
colorDepth.script_url.nunique().compute()

11294

In [19]:
pixelDepth  = df[df.symbol == 'window.screen.pixelDepth']
pixelDepth.head()


,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__34,window.screen.pixelDepth,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/s-code-contents-69cce4f42ae039e7c3974e2fa4292c5e7ad592ab.js,2017-12-16 02:54:21.658,24,2,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__35,window.screen.pixelDepth,https://www.canada.ca/en/services.html,https://assets.adobedtm.com/caacec67651710193d2331efef325107c23a0145/s-code-contents-69cce4f42ae039e7c3974e2fa4292c5e7ad592ab.js,2017-12-16 02:54:21.658,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__381,window.screen.pixelDepth,https://www.coches.net/fiat/segunda-mano/,https://tag.aticdn.net/561882/smarttag.js,2017-12-16 23:20:05.404,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__401,window.screen.pixelDepth,https://www.coches.net/fiat/segunda-mano/,https://tags.tiqcdn.com/utag/schibsted/coches.net/prod/utag.81.js?utv=201712050926,2017-12-16 23:20:05.749,24,2,get,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__402,window.screen.pixelDepth,https://www.coches.net/fiat/segunda-mano/,https://tags.tiqcdn.com/utag/schibsted/coches.net/prod/utag.81.js?utv=201712050926,2017-12-16 23:20:05.749,24,2,get,False


window.navigator.product
The product property returns the engine (product) name of the browser.

Note: This property may not work as expected: All browsers returns "Gecko".

Note: This property is read-only.



In [20]:
#The geolocation property returns a Geolocation object that can be used to locate the user's position.
#Since this can compromise user privacy, the position is not available unless the user approves it.

geolocation = df[df.symbol == 'window.navigator.geolocation']
geolocation.head()


,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_001a6a5225be55c11b9631bdc89cc9836faddb3ff6a3bd1b2280e6e7.json__31,window.navigator.geolocation,https://www.citilink.ru/catalog/for_gamers/igrovye_komplektujushhie/igrovye_videokarty/,https://static.citilink.ru/build/js/commons.bundle.js?1513352112,2017-12-16 23:09:54.054,{},2,get,False
1_001a6a5225be55c11b9631bdc89cc9836faddb3ff6a3bd1b2280e6e7.json__32,window.navigator.geolocation,https://www.citilink.ru/catalog/for_gamers/igrovye_komplektujushhie/igrovye_videokarty/,https://static.citilink.ru/build/js/commons.bundle.js?1513352112,2017-12-16 23:09:54.055,{},2,get,False
1_00219f1af28501c113d2e89bac440951a8247e26746dada9e1ffe509.json__70,window.navigator.geolocation,https://www.bunnings.com.au/our-range/brands/a/aeg,https://www.bunnings.com.au/assets/styleguide/v1.0.884/bunnings-main-site/assets/js/siteJs.min.js,2017-12-16 03:37:55.586,{},2,get,False
1_002844e65d82bb6f49dde43a4bc2a641281f9fd1c2fb7382e285a2d8.json__83,window.navigator.geolocation,http://www.elcorteingles.es/moda/accesorios/estuches-y-neceseres/?level=1,http://www.elcorteingles.es/akam/10/240f2be0,2017-12-16 00:23:38.292,{},2,get,False
1_002844e65d82bb6f49dde43a4bc2a641281f9fd1c2fb7382e285a2d8.json__84,window.navigator.geolocation,http://www.elcorteingles.es/moda/accesorios/estuches-y-neceseres/?level=1,http://www.elcorteingles.es/akam/10/240f2be0,2017-12-16 00:23:38.293,{},2,get,False


In [41]:
geolocation.script_url.nunique().compute()

1359

In [21]:
toDataURL  = df[df.symbol == 'HTMLCanvasElement.toDataURL']
toDataURL.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__260,HTMLCanvasElement.toDataURL,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.499,,0,call,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__287,HTMLCanvasElement.toDataURL,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.505,,0,call,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__289,HTMLCanvasElement.toDataURL,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.707,,0,call,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__613,HTMLCanvasElement.toDataURL,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,http://static.mvideo.ru/media/js/mvideo.js?v0.7,2017-12-16 05:25:25.160,,0,call,False
1_001fa1d1c24cb81d0f954688f75d001122a63bdd4a2eab0ba4c8b4b6.json__74,HTMLCanvasElement.toDataURL,https://reg.gome.com.cn/register/index/person,https://misc.gomein.net.cn/js/regfront/gfp.js,2017-12-16 10:30:48.066,,0,call,False


In [43]:
toDataURL.script_url.nunique().compute()

3441

In [22]:
getImageData = df[df.symbol == 'CanvasRenderingContext2D.getImageData']
getImageData.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00064a670368e73a1ab2afd7576d04973246a0d510a30b08cf5d3ade.json__29,CanvasRenderingContext2D.getImageData,https://torrentsgroup.com/torrents.php?cat=49,https://n-cdn.areyouahuman.com/play/ZQp6LCe0OO3LeZB6ES1CZrJvMefQTtT9oZjddBS5?AYAH_P2=2DE7B66B38AF355AB51D6C2A02AF7BAA&AYAH_F1=Lotame,2017-12-16 23:41:47.106,,0,call,False
1_002844e65d82bb6f49dde43a4bc2a641281f9fd1c2fb7382e285a2d8.json__106,CanvasRenderingContext2D.getImageData,http://www.elcorteingles.es/moda/accesorios/estuches-y-neceseres/?level=1,http://www.elcorteingles.es/akam/10/240f2be0,2017-12-16 00:23:38.816,,0,call,False
1_0082628645aaac3201580397aa4ad4f17a1bce4b8c1feabc1770b080.json__76,CanvasRenderingContext2D.getImageData,http://www.coupang.com/np/categories/335326,http://www.coupang.com/akam/10/1ebb4c57,2017-12-15 22:13:51.626,,0,call,False
1_00a19f92d002b625f2897f9d146cd2995209b96c98556b05489995b2.json__78,CanvasRenderingContext2D.getImageData,http://www.coupang.com/np/categories/328666,http://www.coupang.com/akam/10/93fc4b1,2017-12-16 21:10:52.438,,0,call,False
1_00cbaf3aff9503727aa66f83d7edcd1947e9a0d81e335bd6ec40cf4d.json__537,CanvasRenderingContext2D.getImageData,http://www.cracked.com/article_25101_7-stories-so-horrifying-they-actually-got-cut-from-bible.html,http://imasdk.googleapis.com/js/sdkloader/ima3.js,2017-12-16 19:38:07.147,,0,call,True


In [45]:
getImageData.script_url.nunique().compute()

3441

In [23]:
fillText  = df[df.symbol == 'CanvasRenderingContext2D.fillText']
fillText.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__269,CanvasRenderingContext2D.fillText,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.501,,0,call,False
1_00022de65942197e3855f68b373c5a2710eefe66945e40063c30419f.json__272,CanvasRenderingContext2D.fillText,https://www.coches.net/fiat/segunda-mano/,https://www.coches.net/ztkieflaaxcvaiwh121837.js,2017-12-16 23:20:04.502,,0,call,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__355,CanvasRenderingContext2D.fillText,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,http://static.mvideo.ru/media/js/mvideo.js?v0.7,2017-12-16 05:25:25.098,,0,call,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__356,CanvasRenderingContext2D.fillText,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,http://static.mvideo.ru/media/js/mvideo.js?v0.7,2017-12-16 05:25:25.100,,0,call,False
1_0004d550953191c5723947694ac872f7e277d94041ae0112ffa577fd.json__357,CanvasRenderingContext2D.fillText,http://www.mvideo.ru/aksessuary-dlya-avtomobilei/drugie-avtomobilnye-aksessuary-259/f/category=organaizery-dlya-avtomobilya-1053,http://static.mvideo.ru/media/js/mvideo.js?v0.7,2017-12-16 05:25:25.100,,0,call,False


In [49]:
fillText.script_url.nunique().compute()

3194

## Combining the scripts in one dataframe

In [24]:
candidate=dd.concat([ua,language,platform,cookieenabled,donottrack,localStorage,sessionStorage,plugins,mimeTypes,colorDepth,
            pixelDepth,geolocation,toDataURL,getImageData,fillText],axis=0,interleave_partitions=True)
candidate.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00006011493ed94fb8010cead84ee610cdbece5de961abe96f0cba54.json__2,window.navigator.userAgent,https://vk.com/widget_comments.php?app=2297596&width=100%25&_ver=1&limit=10&height=0&mini=auto&norealtime=0&page=0&status_publish=0&attach=*&url=https%3A%2F%2Fwww.gl5.ru%2Flena-katina-silent-hills.html&title=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&description=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&image=&startWidth=600&referrer=&16060b4b60d,https://vk.com/js/al/aes_light.js?592436914,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_00006b1ad6ce7a5b2b81a8ef96f0778853ea6d81a97693d85a52790c.json__0,window.navigator.userAgent,https://pos.baidu.com/s?hei=70&wid=670&di=u3130587&ltu=https%3A%2F%2Fielts.liuxue86.com%2Fi%2F3040174.html&dis=0&pss=1353x4446&dri=0&ccd=24&ant=0&pis=-1x-1&cce=true&ti=%E5%88%A9%E7%94%A8%E7%B2%BE%E8%AF%BB%E5%88%86%E6%9E%90%E9%9B%85%E6%80%9D%E9%98%85%E8%AF%BB%E6%AE%B5%E8%90%BD&tpr=1513383844066&exps=111000&cfv=0&par=1366x768&dtm=HTML_POST&psr=1366x768&ps=3274x176&ari=2&cja=false&chi=1&prot=2&dai=8&tcn=1513383844&cec=UTF-8&drs=3&cpl=1&dc=2&cdo=-1&tlm=1513383844&pcs=1353x697&cmi=2&col=en-US,https://cpro.baidustatic.com/cpro/ui/noexpire/js/4.0.0/adClosefeedbackUpgrade.min.js,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__43,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.721,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__46,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.722,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__68,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:22.020,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False


In [ ]:
candidate.script_url.nunique().compute()

As we know hs-anaytics, akam, fingerprintjs scripts are browser attribute fingerprinting scripts, excluding them from the dataframe for the time being to focus on othr scripts.(This is done only to make the dataset concise. These scripts will be added back in the final heuristic )

In [25]:
candidate_r=candidate[~candidate.script_url.str.contains('hs-analytic') |~candidate.script_url.str.contains('/akam/') | ~candidate.script_url.str.contains('fingerprint2')  ]
candidate_r.head()

,symbol,location,script_url,time_stamp,value,value_len,operation,in_iframe
call_id,,,,,,,,
1_00006011493ed94fb8010cead84ee610cdbece5de961abe96f0cba54.json__2,window.navigator.userAgent,https://vk.com/widget_comments.php?app=2297596&width=100%25&_ver=1&limit=10&height=0&mini=auto&norealtime=0&page=0&status_publish=0&attach=*&url=https%3A%2F%2Fwww.gl5.ru%2Flena-katina-silent-hills.html&title=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&description=Lena%20Katina%20-%20Silent%20Hills%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%20%D0%BF%D0%B5%D1%81%D0%BD%D0%B8(%D1%81%D0%BB%D0%BE%D0%B2%D0%B0)&image=&startWidth=600&referrer=&16060b4b60d,https://vk.com/js/al/aes_light.js?592436914,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_00006b1ad6ce7a5b2b81a8ef96f0778853ea6d81a97693d85a52790c.json__0,window.navigator.userAgent,https://pos.baidu.com/s?hei=70&wid=670&di=u3130587&ltu=https%3A%2F%2Fielts.liuxue86.com%2Fi%2F3040174.html&dis=0&pss=1353x4446&dri=0&ccd=24&ant=0&pis=-1x-1&cce=true&ti=%E5%88%A9%E7%94%A8%E7%B2%BE%E8%AF%BB%E5%88%86%E6%9E%90%E9%9B%85%E6%80%9D%E9%98%85%E8%AF%BB%E6%AE%B5%E8%90%BD&tpr=1513383844066&exps=111000&cfv=0&par=1366x768&dtm=HTML_POST&psr=1366x768&ps=3274x176&ari=2&cja=false&chi=1&prot=2&dai=8&tcn=1513383844&cec=UTF-8&drs=3&cpl=1&dc=2&cdo=-1&tlm=1513383844&pcs=1353x697&cmi=2&col=en-US,https://cpro.baidustatic.com/cpro/ui/noexpire/js/4.0.0/adClosefeedbackUpgrade.min.js,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,True
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__43,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.721,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__46,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:21.722,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False
1_0000e6f79b14876a8c7ddfcb7782502f2aa02e7803782d46f3bbfea7.json__68,window.navigator.userAgent,https://www.canada.ca/en/services.html,https://www.google-analytics.com/analytics.js,2017-12-16 02:54:22.020,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68,get,False


In [ ]:
candidate_r.script_url.nunique().compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:47818 remote=tcp://127.0.0.1:39769>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:47830 remote=tcp://127.0.0.1:39769>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:47820 remote=tcp://127.0.0.1:39769>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:47826 remote=tcp://127.0.0.1:39769>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:47828 remote=tcp://127.0.0.1:39769>


### Future work

This is an initial analysis, a lot of work needs to be done in order to come up with a definitive heuristic. 

- The symbols considered here are taken from the 3 scripts, hs-analytics, akam, fingerprintjs, other symbols are to be explored.

- Other columns in the dataset need to be analysed apart from the symbols such as the argument column.

- There can be instances of false positves, which have to be found out.
